This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

[(-0.4, 0.010000000000000009), (-0.010000000000000009, 0.4)]
[(0.0, 2.1)]


# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.1, # 1% 
)
#    density=.10, # 1% 

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = Qallse # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
#random_seed=240834366
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834351, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp56ugpakp


08:12:02.495 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
08:12:02.524 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
08:12:02.574 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
08:12:02.609 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/5484
08:12:02.820 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=5484
08:12:02.821 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=818
08:12:02.821 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=60
08:12:02.822 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=1303
08:12:02.822 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834351
08:12:02.823 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2020-02-14T08:12:02.820674
08:12:06.176 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=100811) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp56ugpak

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 5484,
    "num_tracks": 818,
    "num_important_tracks": 60,
    "num_noise": 1303,
    "random_seed": 240834351,
    "time": "2020-02-14T08:12:02.820674",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp56ugpakp",
        "density": 0.1,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834351,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 100811 doublets.
  Input precision (%): 0.4427, recall (%): 98.1982
    New precision (%): 0.4508


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
sliceContainer = model.to_qubo()

08:12:07.753 DEBUG hepqpr.qallse.qallse: using config:
08:12:07.756 DEBUG hepqpr.qallse.qallse:     cheat: False
08:12:07.757 DEBUG hepqpr.qallse.qallse:     max_layer_span: 2
08:12:07.757 DEBUG hepqpr.qallse.qallse:     num_multiplier: -1
08:12:07.758 DEBUG hepqpr.qallse.qallse:     qplet_max_dcurv: 0.0001
08:12:07.759 DEBUG hepqpr.qallse.qallse:     qplet_max_strength: -0.2
08:12:07.760 DEBUG hepqpr.qallse.qallse:     qubo_bias_weight: 0
08:12:07.760 DEBUG hepqpr.qallse.qallse:     qubo_conflict_strength: 1
08:12:07.761 DEBUG hepqpr.qallse.qallse:     rz_power: 1
08:12:07.762 DEBUG hepqpr.qallse.qallse:     strength_bounds: None
08:12:07.763 DEBUG hepqpr.qallse.qallse:     tplet_max_curv: 0.0008
08:12:07.764 DEBUG hepqpr.qallse.qallse:     tplet_max_drz: 0.1
08:12:07.765 DEBUG hepqpr.qallse.qallse:     volayer_power: 2
08:12:07.765 DEBUG hepqpr.qallse.qallse:     xy_power: 1
08:12:07.766 DEBUG hepqpr.qallse.qallse:     xy_relative_strength: 0.5
08:12:10.073 INFO hepqpr.qallse.qallse:

CPU times: user 12 s, sys: 121 ms, total: 12.1 s
Wall time: 12.3 s


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
responseContainer = model.sample_qubo_slices(Q=sliceContainer)
#response = model.sample_qubo_slices(Q=qslice)

{('24130_32265_39523', '24130_32265_39523'): 0, ('93755_109772_114729', '93755_109772_114729'): 0, ('30662_44651_76095', '30662_44651_76095'): 0, ('93719_109735_114944', '93719_109735_114944'): 0, ('93883_109893_114832', '93883_109893_114832'): 0, ('94647_110567_115707', '94647_110567_115707'): 0, ('93850_109576_114807', '93850_109576_114807'): 0, ('94004_110004_115169', '94004_110004_115169'): 0, ('93917_109930_115112', '93917_109930_115112'): 0, ('94227_110199_115361', '94227_110199_115361'): 0, ('19754_27864_40680', '19754_27864_40680'): 0, ('19754_27864_34768', '19754_27864_34768'): 0, ('94351_110306_115465', '94351_110306_115465'): 0, ('28666_36803_43176', '28666_36803_43176'): 0, ('94374_110622_115778', '94374_110622_115778'): 0, ('19754_34768_40680', '19754_34768_40680'): 0, ('94478_110418_115566', '94478_110418_115566'): 0, ('28666_43176_74582', '28666_43176_74582'): 0, ('94441_110095_115249', '94441_110095_115249'): 0, ('30707_37425_44205', '30707_37425_44205'): 0, ('94568_110

08:12:26.296 INFO hepqpr.qallse.qallse: QUBO of size 7829 sampled in 6.22s (seed 523592239).


CPU times: user 5.99 s, sys: 231 ms, total: 6.22 s
Wall time: 6.27 s


## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample_slices(responseContainer=responseContainer)
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

08:12:26.317 INFO hepqpr.qallse.track_recreater: Found 0 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
#en0 = dw.compute_energy(Q)
#en = response.record.energy[0]

#print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
#occs = response.record.num_occurrences
#print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SCORE  -- precision (%): 96.81818181818181, recall (%): 95.94594594594594, missing: 18
          tracks found: 99, trackml score (%): 96.43938842481788


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw,final_doublets,missings, dims=list('xy'))

In [16]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [15]:
#iplot_results(dw, final_doublets, missings, dims=list('zxy'))

In [13]:
iplot_results(dw, final_doublets, missings, dims=list('zy'))